不能将可变参数作为默认值，在创建对象的时候也要时刻小心浅拷贝问题

In [1]:
# 函数可能会修改接收到的任何可变对象
# 如下，不改变数值和元组类型，改变了list类型

def f(a, b):
    a += b
    return a

x = 1
y = 2
f(x, y)

3

In [2]:
x, y

(1, 2)

In [3]:
a = [1, 2]
b = [3, 4]

f(a, b)

[1, 2, 3, 4]

In [4]:
a, b

([1, 2, 3, 4], [3, 4])

In [5]:
t = (10, 20)
u = (40, 50)
f(t, u)

(10, 20, 40, 50)

In [6]:
t, u

((10, 20), (40, 50))

### 8.4.1 不要使用可变类型作为参数的默认值

In [7]:
class HauntedBus:
    """备受幽灵乘客折磨的校车"""
    # 如果没传入passengers参数，使用默认绑定的列表对象，一开始是空列表。
    def __init__(self, passengers=[]):
        # 这个赋值语句把self.passengers变成passengers的别名，而没有传入passengers参数时，后者又是默认列表的别名。
        self.passengers = passengers
    #  在self.passengers上调用.remove()和.append()方法时，修改的其实是默认列表，它是函数对象的一个属性。
    def pick(self, name):
        self.passengers.append(name)
    def drop(self, name):
        self.passengers.remove(name)

In [8]:
bus1 = HauntedBus(['xiaoA', 'xiaoB'])
bus1.passengers

['xiaoA', 'xiaoB']

In [9]:
# 这里可以看到bus1还是很正常的
bus1.pick('xiaoC')
bus1.drop('xiaoA')
bus1.passengers

['xiaoB', 'xiaoC']

In [10]:
# bus2和bus3一开始是默认值，但是bus2指向了默认值所在的地址并且修改了
# 这样一来，后续的默认值都会改变
bus2 = HauntedBus()
bus2.pick('xiaoC')
bus2.passengers

['xiaoC']

In [12]:
# 这里bus3指向了默认指，但是bus2也是默认值，因此出现bug
bus3 = HauntedBus()
bus3.passengers

['xiaoC']

In [13]:
dir(HauntedBus.__init__)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [14]:
HauntedBus.__init__.__defaults__

(['xiaoC'],)

In [15]:
HauntedBus.__init__.__defaults__[0] is bus2.passengers

True

### 8.4.2 防御可变参数

In [16]:
class TwilightBus:
    """让乘客销声匿迹的校车"""
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            # ! 这里复制引用，存在风险
            self.passengers = passengers
    def pick(self, name):
        self.passengers.append(name)
    def drop(self, name):
        self.passengers.remove(name)

# 一开始有5个人
basketball_team = ['Sue', 'Tina', 'Maya', 'Diana', 'Pat']
bus = TwilightBus(basketball_team)
# 下来了两个人
bus.drop('Tina')
bus.drop('Pat')
# 但是车里的人少了的同时，篮球队的人也少了，说明进行了浅拷贝
basketball_team

['Sue', 'Maya', 'Diana']